In [46]:
import pandas as pd
import numpy as np 
import math 
from sklearn.ensemble import RandomForestClassifier


In [47]:
dp = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# age.describe() #decribe column for statiscial value
Y = dp['Survived'].copy()
X = dp[dp.columns.difference(['Survived'])].copy()

In [48]:
X.describe()
# Age has missing data. We fix it by filling missing data with mean

,Age,Fare,Parch,PassengerId,Pclass,SibSp
count,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,32.204208,0.381594,446.000000,2.308642,0.523008
std,14.526497,49.693429,0.806057,257.353842,0.836071,1.102743
min,0.420000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,20.125000,7.910400,0.000000,223.500000,2.000000,0.000000
50%,28.000000,14.454200,0.000000,446.000000,3.000000,0.000000
75%,38.000000,31.000000,0.000000,668.500000,3.000000,1.000000
max,80.000000,512.329200,6.000000,891.000000,3.000000,8.000000


In [49]:
combined_data_set = [X,test]

In [50]:
# female is 0; male is 1
for data_set in combined_data_set:
    data_set['Sex'] = data_set['Sex'].replace({'male':1,'female':0})

In [51]:
for data_set in combined_data_set:
    #new entry for passengers embarked at S
    data_set['S'] = np.where(data_set['Embarked'] == 'S', 1,0)
    #new entry for passengers embarked at C
    data_set['C'] = np.where(data_set['Embarked'] == 'C', 1,0)
    #new entry for passengers embarked at Q
    data_set['Q'] = np.where(data_set['Embarked'] == 'Q', 1,0)
    #drop the remaining 'Embarked' column
    data_set.drop('Embarked',axis = 1,inplace = True)

In [52]:
# Embarked: S is 0; C is 1; Q is 2
#for data_set in combined_data_set:
    #data_set['Embarked'] = data_set['Embarked'].replace({'S':0,'C':1,'Q':2})
    #data_set['Embarked'] = data_set['Embarked'].fillna(1)

In [53]:
# using feature extraction to convert Name into a new feature called 'Titles'
for data_set in combined_data_set:
    data_set['Titles'] = data_set['Name'].str.extract(' ([A-Za-z]+)\.', expand= False)

In [54]:
# we created a new feature called Titles: Mr: 0; Miss: 1; Mrs: 2; Others: 3
titles_mapping = {'Mr': 0, 'Miss': 1, 'Mrs':2}
for data_set in combined_data_set:
    data_set['Titles'] = data_set['Titles'].map(titles_mapping)
    data_set['Titles'] = data_set['Titles'].fillna(3)

In [55]:
#X.groupby('Titles')['Age'].transform('max')
#k = pd.DataFrame({'k':[1,2,3,4,5,6], 'b': ['a','a','b','b','b','b'], 'g':[None,1,1, None,3,4]})
#k['g'].fillna(k.groupby('b')['g'].transform('median'), inplace = True)

In [56]:
# fill the missing ages with median based on their title and normalize age
for data_set in combined_data_set:
    data_set['Age'].fillna(data_set.groupby('Titles')['Age'].transform('median'), inplace = True)
    data_age =(data_set['Age']-data_set['Age'].min())/(data_set['Age'].max()-data_set['Age'].min())
    data_set['Age'] = data_age

In [57]:
#creating spearate entries for all titles
for data_set in combined_data_set:
    #new entry for titles starts with Mr
    data_set['Mr'] = np.where(data_set['Titles'] == 0, 1,0)
    #new entry for titles starts with Miss
    data_set['Miss'] = np.where(data_set['Titles'] == 1, 1,0)
    #new entry for titles starts with Mrs
    data_set['Mrs'] = np.where(data_set['Titles'] == 2, 1,0)
    #new entry for other titles
    data_set['Others'] = np.where(data_set['Titles'] == 3, 1,0)
    #drop the remaining 'Embarked' column
    data_set.drop('Titles',axis = 1,inplace = True)

In [58]:
IDs = test['PassengerId']
# normalize Parch, SibSp and PassengerId
for data_set in combined_data_set:
      data_Parch =(data_set['Parch']-data_set['Parch'].min())/(data_set['Parch'].max()-data_set['Parch'].min())
      data_set['Parch'] = data_Parch
      data_SibSp =(data_set['SibSp']-data_set['SibSp'].min())/(data_set['SibSp'].max()-data_set['SibSp'].min())
      data_set['SibSp'] = data_SibSp
      #data_PassengerId =(data_set['PassengerId']-data_set['PassengerId'].min())/(data_set['PassengerId'].max()-data_set['PassengerId'].min())
      #data_set['PassengerId'] = data_PassengerId

In [59]:
# fill the missing Fare based on the Passanger Class
for data_set in combined_data_set:
    data_set['Fare'].fillna(data_set.groupby('Pclass')['Fare'].transform('median'), inplace = True)
for data_set in combined_data_set:
    data_set.loc[data_set['Fare'] <= 17, 'Fare'] = 0    
    data_set.loc[data_set['Fare'] > 17 & (data_set['Fare'] <= 30),'Fare'] = 1 
    data_set.loc[data_set['Fare'] > 30 & (data_set['Fare'] <= 100),'Fare'] = 2
    data_set.loc[data_set['Fare'] > 100 ,'Fare'] = 3
    #new entry for Cheapest Fare
    data_set['0_Fare'] = np.where(data_set['Fare'] == 0, 1,0)
    #new entry for Second Cheapest Fare
    data_set['1_Fare'] = np.where(data_set['Fare'] == 1, 1,0)
    #new entry for Third Cheapest Fare
    data_set['2_Fare'] = np.where(data_set['Fare'] == 2, 1,0)
    #new entry for Forth Cheapest Fare
    data_set['3_Fare'] = np.where(data_set['Fare'] == 3, 1,0)
    #drop the remaining 'Embarked' column
    data_set.drop('Fare',axis = 1,inplace = True)

In [60]:
#new entries for Pclass
for data_set in combined_data_set:
    #new entry for passengers holds 1st class
    data_set['1_class'] = np.where(data_set['Pclass'] == 1, 1,0)
    #new entry for passengers hold  2nd class
    data_set['2_class'] = np.where(data_set['Pclass'] == 2, 1,0)
    #new entry for passengers holds 3rd class
    data_set['3_class'] = np.where(data_set['Pclass'] == 3, 1,0)
    #drop the remaining 'Embarked' column
    data_set.drop('Pclass',axis = 1,inplace = True)

In [61]:
#normalize the fare column
#for data_set in combined_data_set:
    #data_Fare =(data_set['Fare']-data_set['Fare'].min())/(data_set['Fare'].max()-data_set['Fare'].min())
    #data_set['Fare'] = data_Fare

In [62]:
# convert Cabin to numerical data that we can use

In [63]:
for data_set in combined_data_set:
    data_set.drop(['Cabin','Ticket','Name','PassengerId'], axis = 1, inplace = True)
    #data_set.drop(data_set.columns.difference(['Fare','Age','Sex','Parch']), axis = 1, inplace = True)

In [64]:
model = RandomForestClassifier(n_estimators = 100)
model.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(Y, model.predict(X))

0.94388327721661058

In [66]:
prediction = model.predict(test)
prediction = {'PassengerId': IDs, 'Survived': prediction}
pd.DataFrame(prediction).to_csv('prediction_2.csv', index = False)

In [67]:
X

,Age,Parch,Sex,SibSp,S,C,Q,Mr,Miss,Mrs,Others,0_Fare,1_Fare,2_Fare,3_Fare,1_class,2_class,3_class
0,0.271174,0.000000,1,0.125,1,0,0,1,0,0,0,1,0,0,0,0,0,1
1,0.472229,0.000000,0,0.125,0,1,0,0,0,1,0,0,0,1,0,1,0,0
2,0.321438,0.000000,0,0.000,1,0,0,0,1,0,0,1,0,0,0,0,0,1
3,0.434531,0.000000,0,0.125,1,0,0,0,0,1,0,0,0,1,0,1,0,0
4,0.434531,0.000000,1,0.000,1,0,0,1,0,0,0,1,0,0,0,0,0,1
5,0.371701,0.000000,1,0.000,0,0,1,1,0,0,0,1,0,0,0,0,0,1
6,0.673285,0.000000,1,0.000,1,0,0,1,0,0,0,0,0,1,0,1,0,0
7,0.019854,0.166667,1,0.375,1,0,0,0,0,0,1,0,0,1,0,0,0,1
8,0.334004,0.333333,0,0.000,1,0,0,0,0,1,0,1,0,0,0,0,0,1
9,0.170646,0.000000,0,0.125,0,1,0,0,0,1,0,0,0,1,0,0,1,0
